In [1]:
from pettingzoo.mpe import simple_tag_v2
from gymnasium.utils.save_video import save_video

import time

In [15]:
env = simple_tag_v2.env(num_good=1, 
                        num_adversaries=3, 
                        num_obstacles=2, 
                        max_cycles=100, 
                        continuous_actions=True, 
                        render_mode="human"
)

In [16]:
env.reset()
for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()
    action = None if termination or truncation else env.action_space(agent).sample()  # this is where you would insert your policy
    
    env.step(action)
    time.sleep(0.05)

env.render()
env.close()

todo: 
1. save videos [reference](https://github.com/fireofearth/cpsc533v_project/blob/c897af30921e13ac81ce01cc2a8066e1d46ecc31/colin/evaluate_tag_single_adversary.py)
3. try DQN
4. how to implement multiple network and train on that.


## Action, observation and State

In [14]:
s = env.reset()

# action space with four directions and stay
# [no_action, move_left, move_right, move_down, move_up]
print(env.action_space(agent))

i = 0
# sample for different agents
for agent in env.agent_iter():
    a = env.action_space(agent).sample()
    print(agent, ':')
    env.step(a)
    print('sample action:', a)
    # print(env.observation_spaces)
    
    i = i + 1
    if i == 4:
        break

env.close()

Box(0.0, 1.0, (5,), float32)
adversary_0 :
sample action: [0.0119764  0.252828   0.18079923 0.7381237  0.6851406 ]
adversary_1 :
sample action: [0.10676163 0.53216404 0.11538256 0.9378003  0.21284239]
adversary_2 :
sample action: [0.01342471 0.95402396 0.5021012  0.8891766  0.65971637]
agent_0 :
sample action: [0.19853032 0.1248109  0.7256598  0.19498755 0.21039023]


In [82]:
# state, should be 62?
env.state()

array([ 0.0000000e+00,  0.0000000e+00, -8.5962892e-01,  8.8099217e-01,
        1.0135393e+00, -4.1221848e-01,  3.3933982e-01, -1.7276236e+00,
        9.8698556e-01, -9.8570061e-01,  8.6008966e-01, -1.6648449e-01,
        1.0217097e+00, -1.6673965e+00,  0.0000000e+00,  4.0000001e-01,
       -3.0000001e-01, -0.0000000e+00,  1.2735660e-01, -1.0470843e-01,
        2.6553761e-02,  5.7348216e-01, -6.4764571e-01, -7.4192303e-01,
       -9.8698556e-01,  9.8570061e-01, -1.2689586e-01,  8.1921613e-01,
        3.4724202e-02, -6.8169594e-01,  0.0000000e+00,  4.0000001e-01,
       -5.2957845e-01,  4.5698982e-01,  4.6074370e-04,  7.1450770e-01,
        1.5344962e-01, -2.4573398e-01, -5.2074987e-01, -1.5611391e+00,
       -8.6008966e-01,  1.6648449e-01,  1.2689586e-01, -8.1921613e-01,
        1.6162007e-01, -1.5009121e+00,  0.0000000e+00,  4.0000001e-01,
        0.0000000e+00,  4.0000001e-01,  1.6208081e-01, -7.8640437e-01,
       -8.1704417e-03,  1.2551781e+00, -6.8236995e-01, -6.0227096e-02,
      

In [83]:
s = env.reset()

i = 0
for agent in env.agent_iter():

    print(agent, ':')
    env.step(env.action_space(agent).sample())
    # observation space: shape(14),(16)
    # Agent and adversary observations: [self_vel, self_pos, landmark_rel_positions, other_agent_rel_positions, other_agent_velocities]
    print(env.observe(agent))
    a = env.observe(agent)
    
    i = i + 1
    if i == 4:
        break

env.close()

adversary_0 :
[ 0.          0.         -0.34912184  0.5028731  -0.44968572 -0.1900941
  0.9063153  -0.38171014 -0.6211034   0.0766574   0.72543776 -0.48805845
  1.1138225  -1.4590774   0.          0.        ]
adversary_1 :
[ 0.          0.         -0.9702253   0.57953054  0.17141771 -0.2667515
  1.5274187  -0.45836753  0.6211034  -0.0766574   1.3465412  -0.56471586
  1.7349259  -1.5357348   0.          0.        ]
adversary_2 :
[ 0.          0.          0.37631595  0.0148147  -1.1751235   0.29796433
  0.18087757  0.10634831 -0.72543776  0.48805845 -1.3465412   0.56471586
  0.38838464 -0.9710189   0.          0.        ]
agent_0 :
[ 0.         -0.4         0.7647006  -0.9962042  -1.5635082   1.3089832
 -0.20750709  1.1173673  -1.0838225   1.4990773  -1.7349259   1.6057347
 -0.47456777  0.9779856 ]


In [84]:
# [self_vel, self_pos, landmark_rel_positions, other_agent_rel_positions, other_agent_velocities]
# all of them has two elements
a.shape

(14,)

## Policy network

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# figure out how many networks
# start from basic code from dqn
# read the paper

In [ ]:
def bound(x):
    if x < 0.9:
        return 0
    if x < 1.0:
        return (x - 0.9) * 10
    return min(np.exp(2 * x - 2), 10)

In [ ]:
class DQN(nn.Module):
    """Deep Q-network with target network"""
    
    def __init__(self, n_inputs, n_outputs, learning_rate):
        super(DQN, self).__init__()
        # network
        self.out = nn.Linear(n_inputs, n_outputs)
        # training
        self.optimizer = optim.SGD(self.parameters(), lr=learning_rate)
    
    def forward(self, x):
        return self.out(x)
    
    def loss(self, q_outputs, q_targets):
        return torch.sum(torch.pow(q_targets - q_outputs, 2))
    
    def update_params(self, new_params, tau):
        params = self.state_dict()
        for k in params.keys():
            params[k] = (1-tau) * params[k] + tau * new_params[k]
        self.load_state_dict(params)